In [1]:
from functions_trained_reservoir import *

from reservoirpy.nodes import Reservoir,FORCE,Ridge
from reservoirpy.mat_gen import Initializer,_scale_spectral_radius
from reservoirpy.observables import spectral_radius
from tqdm import tqdm

from ReccurentNetwork import *

import numpy as np
from numpy.linalg import eig

import spicy as sp
import matplotlib.pyplot as plt

# Réseaux de neurones

On cherche ici dans un premier temps à reprdoduire les figures les plus simples de l'article. 

Les mesures de l'article sont effectuées sur 2 réservoirs : 

  - le premier est dit entrainé : on va alors entrainer le réservoir et la sortie
  - le second lui est dit non entrainé : on entraine seulement la sortie
    

Ce premier script s'intéresse donc à la reproduction de l'entrainement des figures 3, 4 et 5 ainsi qu'aux résultas obtenus sur un réservoir entrainé.
 
La librairie utilisé pour entrainer le réservoir est prise dans autre article ReScience : https://github.com/ReScience-Archives/Vitay-2016

Copyright (c) 2016 Julien Vitay julien.vitay@informatik.tu-chemnitz.de

## Expériences

Deux expériences quasiment similaires sont proposés pour un réservoir totalement entrainé. La seule différence entre ces dernières sont le nombre de neurones utilisés dans le réservoir : 

   - Soit N = 2100 pour des résultats robustes entre toutes les données pour un sujet
   - Soit N = 4000 pour des résultats robustes entre tous les sujets (5 au maximum)
    
Nous traiterons le premier  cas dans ce papier.

##### Méthode d'entrainement

Pour ce qui est de l'entrainement du réservoir nous allons procéder en 2 étapes. En effet, nous séparons notre entrée en deux parties : l'époque sensorielle et l'époque motricielle. Nous séparons l'entrainement de ces 2 époques que je vais détailler ci dessous.

###### Epoque sensorielle : 

Pour un sujet donné et parmit toutes les entrées différentes, pour chaque chiffre on selectionne une entrée moyenne. Cette entrée est injectée dans le réservoir sans bruit et sans target, on obtient alors une trajectoire resultante de dimension (??) pour chaque chiffre. Ces 10 trajectoires sont appelées les "innate trajectories" et seront les targets pour toutes les autres entrées durant l'entrainement du l'époque sensorielle en faisant bien attention à  ce que la dimension d'entrée des nouvelles entrées soit cohérente avec celle de l'innate trajectorie, on procédera alors pas une déformation linéaire de l'innate trajectory. De plus, du bruit sera présent durant l'entrainement.

###### Epoque motrice

target = une target par chiffre peu importe l'entrée ou le sujet
c'est quoi la target ?? 
comprends la transition de 300 ms aussi


###### Test
Les test effectués dans ce script seront fait sur le sujet 1

Données générales 

In [2]:
data = extract_data() #Données d'entrée
transcription = extract_target() #Données d'arrivée 

#Les données suivantes sont utilisés dans les fonctions mais pas dans le script ! 

freq_ent = 6*10**3  #fréquence d'entrée
T_ent = 1/freq_ent  #période d'entrée

freq_out = 10**3 #fréquence de sortie 
T_out = 1/freq_out #période de sortie

ampl_int = 5 #Amplitude d'entrée

Données du réservoir

In [3]:
N = 100 #Nombre de neurones dans le réservoir
M = 12 #Nombres d'entrées
No = 3 #Nombres de sorties
tau = 25*10e-3
dt = 1e-3
lr = dt/tau
sr = 1.6 #spectral radius --> gain ?
g = 1.6 #le gain du réseau ?                             #je comprends pas trop cette donnée. 
pc = 0.2 # connectivité entre les neurones (pc dans l'article)
SD = g/np.sqrt(pc*N)
mu = 0
I0 = 0.05 #Bruit dans le réservoir

Wo = normal(No,N,loc=mu,scale = 1/np.sqrt(N))    #Matrice de sortie : Non utilisée


Création de la matrice du réservoir : avec pas d'autapse
Puis bricolage pour réxupérer le rayon spectral voulu

In [4]:
W_r.init = Initializer(W_r)

W = _scale_spectral_radius(W_r.init, [N,N], sr)
W = W.toarray()

#Vérification du rayon spectral

#a,b=eig(W)
#print(np.max(np.abs(a)))


Création de la matrice d'entrée : chaque entrée k est projetée sur le neurone $(k-1)\frac{N}{M} + 1$ jusqu'au neurone $k\frac{N}{M}$ selon une loi normale 

In [5]:
Win = W_in(N,M)

### Epoque sensorielle 

Récupération des cocleograms avec une taille moyenne pour chaque chiffre 

In [6]:
median_entrance_cocleogram = median_cocleogram(data,sujet=[1])

Création du réservoir

In [7]:
network = RecurrentNetwork(
    W_in = Win,
    W_rec = W,
    W_out = Wo,
    Ni = M, # Number of inputs
    N = N, # Number of recurrent neurons
    No = No, # Number of read-out neurons
    tau = 25, # Time constant of the neurons
    g = g, # Synaptic strength scaling
    pc = pc, # Connection probability
    Io = I0, # Noise variance
    delta = 1.0, # Initial diagonal value of the P matrix                                 # ?
    P_plastic = 0.6, # Percentage of neurons receiving plastic synapses                   # ?
)

Détermination des trajectoires inées  

In [8]:
innate_trajectories_sensor = []
innate_trajectories_motor = []
for i in range(len(median_entrance_cocleogram)):
    
    #Récupération du cocleogram moyen
    impulse = np.expand_dims(median_entrance_cocleogram[i], axis=2)
    
    #Récupération des trajectoires innées de l'époque sensorielle et de l'époque motricielle ???? 
    initial_trajectory, initial_output = network.simulate(stimulus=impulse, noise=False)
    
    #Ensuite on les trie
    innate_trajectories_sensor.append(np.squeeze(initial_trajectory,axis = 2))
    innate_trajectories_motor.append(np.squeeze(initial_output,axis = 2))

100%|█████████████████████████████████████████████████████████████████████████████| 712/712 [00:00<00:00, 16223.52it/s]


#### Entrainement de l'epoque sensorielle.

Récupération des données d'entrées et des targets, ici on entraîne sur un sujet, 3 entrées et les 10 chiffres

Attention : penser à rajouter du bruit !!!!

In [9]:
X_input = []
X_target = []
for indS in [1] : 
    for indU in range(1,4):
        for indD in range(10):
            X_input.append(np.transpose(cocleogram(indS, indU, indD,data)))
            X_target.append(linear_warping(indS,indU,indD,data,innate_trajectories_sensor[indD]))

Entrainement

In [10]:
def _rls(P, x, e):
    """Recursive Least Squares learning rule."""
    k = np.dot(P, x)
    xPx = np.dot(x.T, k)
    c = float(1.0 / (1.0 + xPx))
    P = P - c * np.outer(k, k)
    dw = -c * np.outer(e, k)

    return dw, P


##### Test de RLS sans utiliser le noeud de reservoirpy

In [11]:
P = np.identity(N)                                                  # matrice de gain ?
x = np.random.normal(0,1,(N,1))                                     #etats initiaux du réservoir

for xi,xt in tqdm(zip(X_input,X_target)):                         #entrainement
    for t in range(len(xi)):                                           # pour 1 pas de temps 
        
        x1 = (1-lr)*x + lr*np.tanh(np.dot(Win,np.expand_dims(xi[t], axis=1))+np.dot(W,x))   #calcul de x(t+1)
        
        e = np.expand_dims(xt[t],axis=1) - x1                       #calcul de l'erreur pour un seul pas de temps
        
        
        dW,P = _rls(P, x1, e)                                       #calcul du rls pour un pas de temps 
        
        W -= dW
        


30it [00:04,  6.61it/s]


###### Test de RLS en utilisant le noeud de reservoirpy

In [17]:
reservoir = Reservoir(W=W,Win=Win)
P = np.identity(N)           

for xi,xt in tqdm(zip(X_input,X_target)):
    x1 = reservoir.run(xi)
    e = x1-xt
    for t in range(len(x1)):
        dW, P = _rls(P,x1[t],e[t])
        W -= dW
    reservoir.set_param("W",W)
    

0it [00:00, ?it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 531/531 [00:00<00:00, 1141.90it/s]
1it [00:00,  1.63it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 448/448 [00:00<00:00, 3270.78it/s]
2it [00:00,  2.50it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 529/529 [00:00<00:00, 4676.50it/s]
3it [00:01,  3.25it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 638/638 [00:00<00:00, 5664.94it/s]
4it [00:01,  3.36it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 611/611 [00:00<00:00, 2993.57it/s]
5it [00:01,  3.32it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 688/688 [00:00<00:00, 4975.61it/s]
6it [00:01,  3.55it/s]
Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 761/761 [00:00<00:00, 5948.46it/s]
7it

Running Reservoir-5: 100%|█████████████████████████████████████████████████████████| 698/698 [00:00<00:00, 4050.82it/s]
30it [00:08,  3.60it/s]
